In [3]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

from save_measures import Run
from precomputation import RunPair
from config import ExperimentType
from correlations import initialization_correlation, conditional_correlation, hp_correlation
from plotting import correlation_envelope

In [4]:
def new_correlation(pair_corrs):
    def condition(hp_pair):
        return hp_pair[0][1] == ExperimentType.NO_DROPOUT and hp_pair[1][1] == ExperimentType.NO_DROPOUT

    return conditional_correlation(pair_corrs, condition)

In [7]:
# for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
#           "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
with open("./data.nosync/pre-comp-correlations/pac_bayes-best.pickle", "rb") as f:
    pair_corrs = pickle.load(f)

overall_corrs = new_correlation(pair_corrs)

for x in overall_corrs["weighted"][10]:
    corr = np.mean([a[0] for a in overall_corrs["weighted"][10][x]])
    print("{}: {}".format(x, corr))

pacbayes_flatness: -0.376782106878178
pacbayes_init: -0.4418631341236815
pacbayes_init_full: -0.1817513195780521
pacbayes_init_main: -0.4418631341236815
pacbayes_orig: -0.4526968522565394
pacbayes_orig_full: -0.24150052131896166
pacbayes_orig_main: -0.4526968522565394
pacbayes_mag_flatness: -0.38152173824040525
pacbayes_mag_init: 0.08998269890628292
pacbayes_mag_init_full: 0.12000789194995377
pacbayes_mag_init_main: 0.08998269890628292
pacbayes_mag_orig: 0.06306323379955477
pacbayes_mag_orig_full: 0.11096206041881174
pacbayes_mag_orig_main: 0.06306323379955477


In [43]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation(pair_corrs)
    
    for x in overall_corrs["weighted"][0.01]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][0.01][x]])
        if corr > 0.2 or corr < -0.2:
            print("{}: {}".format(x, corr))

log_prod_of_spec_over_margin_fft: 0.23801403346589886
log_spec_init_main_fft: 0.22891858196865703
log_spec_orig_main_fft: 0.2287747240206565
log_sum_of_spec_over_margin_fft: 0.26309362986270124
path_norm_over_margin: -0.21029887045422826
spec_init: 0.22706811081399716
spec_orig: 0.22718684553774887
log_prod_of_spec_over_exponential_margin_fft: 0.2540135600624056
log_spec_init_main_exponential_margin_fft: 0.24828489789023364
log_spec_orig_main_exponential_margin_fft: 0.24882936865964447
log_sum_of_spec_over_exponential_margin_fft: 0.273269517498165
spec_init_exponential_margin: 0.24613914732601758
spec_orig_exponential_margin: 0.24742652012057356
log_prod_of_spec_over_normalized_exponential_margin_fft: 0.2540135600624056
log_spec_init_main_normalized_exponential_margin_fft: 0.24828489789023364
log_spec_orig_main_normalized_exponential_margin_fft: 0.24882936865964447
log_sum_of_spec_over_normalized_exponential_margin_fft: 0.26534293856552926
spec_init_normalized_exponential_margin: 0.246

In [42]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation(pair_corrs)
    
    for x in overall_corrs["weighted"][0.1]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][0.1][x]])
        if corr > 0.4 or corr < -0.4:
            print("{}: {}".format(x, corr))

margin: -0.4231043988829352
inverse_margin: -0.4231043988829352
fro_over_spec_over_margin_fft: 0.4845372502214156
path_norm_over_margin: -0.5438275431793183
exponential_margin: 0.4231043988829352
path_norm_over_exponential_margin: 0.4881370313020978
normalized_exponential_margin: 0.4231043988829352
path_norm_over_normalized_exponential_margin: 0.4881370313020978
shifted_margin: 0.4231043988829352
path_norm_over_shifted_margin: 0.49161867442852925
pacbayes_init: -0.4053838608154554
pacbayes_init_main: -0.4053838608154554
pacbayes_mag_flatness: -0.4515336671866526
path_norm: -0.5091272860177997
l2_dist: -0.45135574217861707
fro_dist: -0.45135574217861707


In [41]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation(pair_corrs)
    
    for x in overall_corrs["weighted"][1]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][1][x]])
        if corr > 0.5 or corr < -0.5:
            print("{}: {}".format(x, corr))

margin: -0.5142385144380694
inverse_margin: -0.5142385144380694
fro_over_spec_over_margin_fft: 0.5078657071403239
path_norm_over_margin: -0.6491114201565285
exponential_margin: 0.5142385144380694
path_norm_over_exponential_margin: 0.5922120235253293
normalized_exponential_margin: 0.5142385144380694
path_norm_over_normalized_exponential_margin: 0.5922120235253293
shifted_margin: 0.5142385144380694
path_norm_over_shifted_margin: 0.5461703486892373
path_norm: -0.6236360893475605
l2_dist: -0.5450881430537146
fro_dist: -0.5450881430537146
accuracy/test: 0.5766009368803517
accuracy/train: 0.5178614198330163
cross_entropy_epoch_end/train: -0.6730909287336697
cross_entropy_epoch_end/test: -0.6955472926845497
sotl: -0.7343323050973952
sotl-ema: -0.7343323050973952
sotl-1: -0.7343323050973952
sotl-2: -0.7343323050973952
sotl-3: -0.7343323050973952
sotl-5: -0.7343323050973952


In [44]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation(pair_corrs)
    
    for x in overall_corrs["weighted"][5]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][5][x]])
        if corr > 0.5 or corr < -0.5:
            print("{}: {}".format(x, corr))

path_norm_over_margin: -0.5773965207593291
path_norm_over_exponential_margin: 0.7233351706572383
path_norm_over_normalized_exponential_margin: 0.7233351706572383
path_norm_over_shifted_margin: 0.7165864611930595
path_norm: -0.689104825713056
l2_dist: -0.597586597208694
fro_dist: -0.597586597208694
accuracy/test: 0.7656084840810315
cross_entropy_epoch_end/train: -0.5977807291539873
cross_entropy_epoch_end/test: -0.7959960734797523
sotl: -0.7111405527809176
sotl-ema: -0.7037726500670256
sotl-1: -0.6319845032881029
sotl-2: -0.7111405527809176
sotl-3: -0.7111405527809176
sotl-5: -0.7111405527809176


In [45]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation(pair_corrs)
    
    for x in overall_corrs["weighted"][10]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][10][x]])
        if corr > 0.5 or corr < -0.5:
            print("{}: {}".format(x, corr))

path_norm_over_margin: -0.6001678114032326
fro_over_spec_over_exponential_margin_fft: -0.5388246343418008
log_sum_of_fro_over_exponential_margin: -0.6208525622458316
fro_over_spec_over_normalized_exponential_margin_fft: -0.5388246343418008
log_sum_of_fro_over_normalized_exponential_margin: -0.5217541134997687
fro_over_spec_over_shifted_margin_fft: -0.5370420046330171
log_sum_of_fro_over_shifted_margin: -0.5390125563035332
path_norm: -0.6698453969037773
accuracy/test: 0.847550562255253
cross_entropy_epoch_end/test: -0.734899999570688
sotl: -0.6235499890834532
sotl-ema: -0.5884200199965639
sotl-2: -0.6235499890834532
sotl-3: -0.6235499890834532
sotl-5: -0.6235499890834532


In [46]:
def new_correlation2(pair_corrs):
    def condition(hp_pair):
        return hp_pair[0][1] != ExperimentType.BATCH_NORM and hp_pair[1][1] != ExperimentType.BATCH_NORM

    return conditional_correlation(pair_corrs, condition)

In [55]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation2(pair_corrs)
    
    for x in overall_corrs["weighted"][0.001]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][0.001][x]])
        if corr > 0.25 or corr < -0.25:
            print("{}: {}".format(x, corr))

log_spec_init_main_fft: 0.25295527671566354
spec_init: 0.25213965800848964
log_spec_init_main_exponential_margin_fft: 0.25106140218193157
log_spec_init_main_normalized_exponential_margin_fft: 0.25106140218193157
log_spec_init_main_shifted_margin_fft: 0.2532729930121539
spec_init_shifted_margin: 0.251344851962594
pacbayes_init: 0.29683316803485177
pacbayes_init_full: 0.2981208814544369
pacbayes_init_main: 0.29683316803485177
pacbayes_mag_init: 0.27263068440582683
pacbayes_mag_init_full: 0.27263068440582683
pacbayes_mag_init_main: 0.27263068440582683
l2_dist: 0.29681415078861734
fro_dist: 0.29681415078861734
dist_spec_init_fft: 0.3426751683199198


In [56]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation2(pair_corrs)
    
    for x in overall_corrs["weighted"][0.01]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][0.01][x]])
        if corr > 0.25 or corr < -0.25:
            print("{}: {}".format(x, corr))

log_prod_of_spec_over_margin_fft: 0.2744487544309763
log_spec_init_main_fft: 0.2681841680753995
log_spec_orig_main_fft: 0.26834792762277887
log_sum_of_spec_over_margin_fft: 0.2843334469795582
spec_init: 0.2672438178370279
spec_orig: 0.26708923548454666
log_prod_of_spec_over_exponential_margin_fft: 0.2857549376437252
log_spec_init_main_exponential_margin_fft: 0.2809160707183078
log_spec_orig_main_exponential_margin_fft: 0.2817057120848158
log_sum_of_spec_over_exponential_margin_fft: 0.29671189606025145
spec_init_exponential_margin: 0.2794718608450742
spec_orig_exponential_margin: 0.28086627275488557
log_prod_of_spec_over_normalized_exponential_margin_fft: 0.2857549376437252
log_spec_init_main_normalized_exponential_margin_fft: 0.2809160707183078
log_spec_orig_main_normalized_exponential_margin_fft: 0.2817057120848158
log_sum_of_spec_over_normalized_exponential_margin_fft: 0.2925544160665981
spec_init_normalized_exponential_margin: 0.2794718608450742
spec_orig_normalized_exponential_marg

In [53]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation2(pair_corrs)
    
    for x in overall_corrs["weighted"][0.1]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][0.1][x]])
        if corr > 0.3 or corr < -0.3:
            print("{}: {}".format(x, corr))

fro_over_spec_over_margin_fft: 0.32434126773611127
path_norm_over_margin: -0.44212141816667344
path_norm_over_exponential_margin: 0.40677916820496773
path_norm_over_normalized_exponential_margin: 0.40677916820496773
path_norm_over_shifted_margin: 0.41455692322068693
pacbayes_init: -0.3350514400065865
pacbayes_init_full: -0.31280856807175084
pacbayes_init_main: -0.3350514400065865
pacbayes_mag_flatness: -0.38504449920353473
path_norm: -0.41899833634705347
l2_dist: -0.34315259121065095
fro_dist: -0.34315259121065095


In [54]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation2(pair_corrs)
    
    for x in overall_corrs["weighted"][1]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][1][x]])
        if corr > 0.4 or corr < -0.4:
            print("{}: {}".format(x, corr))

path_norm_over_margin: -0.5498736779281601
path_norm_over_exponential_margin: 0.48868565091034044
path_norm_over_normalized_exponential_margin: 0.48868565091034044
path_norm_over_shifted_margin: 0.45677542244073055
path_norm: -0.5115729251427611
l2_dist: -0.4197725209220181
fro_dist: -0.4197725209220181
accuracy/test: 0.43687115396096293
cross_entropy_epoch_end/train: -0.5052397287801514
cross_entropy_epoch_end/test: -0.5269393675679221
sotl: -0.5139840897592217
sotl-ema: -0.5139840897592217
sotl-1: -0.5139840897592217
sotl-2: -0.5139840897592217
sotl-3: -0.5139840897592217
sotl-5: -0.5139840897592217


In [51]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation2(pair_corrs)
    
    for x in overall_corrs["weighted"][5]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][5][x]])
        if corr > 0.5 or corr < -0.5:
            print("{}: {}".format(x, corr))

path_norm_over_exponential_margin: 0.5859548685633952
path_norm_over_normalized_exponential_margin: 0.5859548685633952
path_norm_over_shifted_margin: 0.5807441962277956
path_norm: -0.560582986161712
accuracy/test: 0.5828660598041647
cross_entropy_epoch_end/test: -0.5967506464457738


In [52]:
for a in ["margin", "exponential_margin", "normalized_exponential_margin", "shifted_margin", 
          "pac_bayes", "vc_dim", "norms", "optimisation"]:
    
    with open("./data.nosync/pre-comp-correlations/{}-best.pickle".format(a), "rb") as f:
        pair_corrs = pickle.load(f)
        
    overall_corrs = new_correlation2(pair_corrs)
    
    for x in overall_corrs["weighted"][10]:
        corr = np.mean([a[0] for a in overall_corrs["weighted"][10][x]])
        if corr > 0.5 or corr < -0.5:
            print("{}: {}".format(x, corr))

path_norm_over_margin: -0.5029382374973821
path_norm: -0.5483767405925426
accuracy/test: 0.6669876305738258
cross_entropy_epoch_end/test: -0.5785352102793938


In [18]:
with open("./data.nosync/pre-comp-pairs/margin-best.pickle", "rb") as f:
    pairs = pickle.load(f)

In [26]:
a = []

for hp_pair, run_pair_dict in pairs.items():
    if hp_pair[0][1] != ExperimentType.NO_DROPOUT or hp_pair[1][1] != ExperimentType.NO_DROPOUT:
        continue
    
    for seed_pair, run_pair in run_pair_dict.items():
        a.append(run_pair.weight * run_pair.errors[0.001]["log_spec_init_main_fft"])

In [27]:
sum(a)/sum([abs(x) for x in a])

0.2302234848509048

In [37]:
with open("./data.nosync/runs/no_dropout.pickle", "rb") as f:
    runs = pickle.load(f)

In [45]:
meas = []
acc = []

for hp, l in runs.items():
    for r in l:
        meas.append(r.measures[1]["accuracy/test"])
        acc.append(r.best_test_acc)

In [46]:
len(meas)

384

In [47]:
len(acc)

384

In [48]:
a = []
for s in acc:
    if s < 0.5:
        a.append(s)

In [49]:
len(a)

1

In [50]:
from scipy import stats

In [51]:
stats.kendalltau(meas, acc)

KendalltauResult(correlation=0.504602964871495, pvalue=2.8607114605570436e-49)

In [57]:
with open("./data.nosync/runs/base.pickle", "rb") as f:
    runs = pickle.load(f)

In [ ]:
e = []
i = []
m = []
t = []

for r in runs:
    e.append(r.measures[5]["exponential_margin"])
    i.append(r.measures[5]["inverse_margin"])
    m.append(r.measures[5]["inverse_margin"])